<a href="https://colab.research.google.com/github/aly-such/data-512-a2/blob/main/hcds_a2_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# install ores (run only one time)
# !pip install ores

# imports
import requests
import json
import pandas as pd
import numpy as np
from ores import api

In [10]:
# Politicians by Country dataset - from Figshare
wiki = pd.read_csv('C:\Users\amsuc\OneDrive\Documents\Washington\Autumn 2021\HCDD\A2\page_data.csv')
# Population Data - from Population Reference Bureau
pop = pd.read_csv('C:\Users\amsuc\OneDrive\Documents\Washington\Autumn 2021\HCDD\A2\WPDS_2020_data.csv')

SyntaxError: ignored

In [ ]:
wiki.head()

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409


In [ ]:
# Template:... are not wiki articles, drop from dataframe
wiki = wiki[~wiki.page.str.contains("Template:")]

In [ ]:
wiki

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568
...,...,...,...
47192,Yahya Jammeh,Gambia,807482007
47193,Lucius Fairchild,United States,807483006
47194,Fahd of Saudi Arabia,Saudi Arabia,807483153
47195,Francis Fessenden,United States,807483270


In [ ]:
pop.head()

,FIPS,Name,Type,TimeFrame,Data (M),Population
0,WORLD,WORLD,World,2019,7772.850,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000


In [ ]:
# Separate The regional populations from the country populations
# First, create a dataframe of the Names that are not in all caps (country-level counts)
# These rows will match country values in paga_data.csv
country_pop = pop[~pop.Name.str.isupper()]

# Second, create a dataframe of the Names that are in all caps (regional-level counts)
# These rows will not have a match in paga_data.csv
region_pop = pop[pop.Name.str.isupper()]

In [ ]:
region_pop.head()

,FIPS,Name,Type,TimeFrame,Data (M),Population
0,WORLD,WORLD,World,2019,7772.850,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
10,WESTERN AFRICA,WESTERN AFRICA,Sub-Region,2019,401.115,401115000
27,EASTERN AFRICA,EASTERN AFRICA,Sub-Region,2019,444.970,444970000


In [ ]:
country_pop.head()

,FIPS,Name,Type,TimeFrame,Data (M),Population
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000
5,LY,Libya,Country,2019,6.891,6891000
6,MA,Morocco,Country,2019,35.952,35952000
7,SD,Sudan,Country,2019,43.849,43849000


In [ ]:
ores_session = api.Session

In [ ]:
# Define the endpoint
endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'

# Define header
# Customize these with your own information
headers = {
    'User-Agent': 'https://github.com/ams884',
    'From': 'ams884@uw.edu'
}

# Article quality prediction function
def ores_api_call(rev_ids, headers):
    
    # Define the parameters that will be used in the endpoint
    param = {'project' : 'enwiki',
              'model'   : 'wp10',
              'revids'  : '|'.join(str(x) for x in rev_ids)
              }
    
    # Call API endpoint
    call = requests.get(endpoint.format(**param))
    response = call.json()
    
    # Loop through each revision_id to get its respective prediction score
    for x in rev_ids:
        
        # Turn revision_ids into a string
        item = str(x)
        
        # Append each Prediction Score to self-titled list as loop runs
        prediction.append(response['enwiki']['scores'][item]['wp10']['score']['prediction'])
        
        # Append each revision_id to self-titled list as loop runs
        rev_id.append(x)

In [ ]:
# Call ores_api_call
# It is recommened to run this in 50 revision id batches at a time. This will be implemented within the loop for ease of use

# Create empty lists that will be appended to
prediction = []
rev_id = []

# Start iteration at zero
i=0

# Create a loop that will call ores_api_call and run in batches of 50
while i < len(wiki):
    try:
        ids = []
        j = 0
        while j < 25:
            ids.append(wiki['rev_id'][i])
            j += 1
            i += 1  
        ores_api_call(ids, headers)
    except Exception:
        pass

# Print prediction and revision_id length   
print(len(prediction))
print(len(rev_id))

KeyboardInterrupt: 

In [ ]:
# Merge prediction and rev_id
pred_revid_df = pd.DataFrame({'article_quality':prediction, 'rev_id':revision_id})

# Merge pred_revid_df with wiki 
wiki_pred_df = pred_revid_df.merge(wiki, left_on='rev_id', right_on='rev_id', how='inner')

In [ ]:
wiki_pred_df.head()

In [ ]:
# Merge wiki/pred data with country population data
merge_df = wiki_pred_df.merge(country_pop, left_on='country', right_on='Name', how='inner')

In [ ]:
merge_df.head()

In [ ]:
# Create single dataframe of wikipedia, prediction, and country population data
# This is to simply rename all the columns to intuitive names

wikipedia_df = pd.DataFrame({
    'country' : merge_df['country'],
    'article_name' : merge_df['page'],
    'revision_id' : merge_df['rev_id'],
    'article_quality_est.' : merge_df['article_quality'],
    'population' : merge_df['Population']
} )

In [ ]:
wikipedia_df.to_csv('articles_with_population', sep = ',')

In [ ]:
# Count of Articles by Country
articles_by_country = wikipedia_df.groupby('country').count()['article_name'].astype(int).reset_index()

articles_by_country.head()

In [ ]:
!pip install ores

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\amsuc\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\amsuc\\AppData\\Local\\Temp\\pip-install-fftok2ue\\pyenchant_9b48daae32d44f7ebeb9bdde6f2a1399\\setup.py'"'"'; __file__='"'"'C:\\Users\\amsuc\\AppData\\Local\\Temp\\pip-install-fftok2ue\\pyenchant_9b48daae32d44f7ebeb9bdde6f2a1399\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\amsuc\AppData\Local\Temp\pip-pip-egg-info-aip5rkl8'
         cwd: C:\Users\amsuc\AppData\Local\Temp\pip-install-fftok2ue\pyenchant_9b48daae32d44f7ebeb9bdde6f2a1399\
    Complete output (9 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\amsuc\AppData\Local\Temp\pip-install-fftok2ue\pyenchant_9b48daae32d44f7ebeb9bdde6f2a1399\setup.


  Using cached ores-1.4.0-py2.py3-none-any.whl (160 kB)
  Using cached redis-3.5.3-py2.py3-none-any.whl (72 kB)
  Using cached flask_swaggerui-0.0.1-py2.py3-none-any.whl (740 kB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Using cached Flask-Jsonpify-1.5.0.tar.gz (3.0 kB)
  Using cached flask_wikimediaui-0.0.1-py2.py3-none-any.whl (2.5 MB)
  Using cached PyYAML-4.2b4.tar.gz (262 kB)
  Using cached Werkzeug-0.16.1-py2.py3-none-any.whl (327 kB)
  Using cached Flask-1.0.4-py2.py3-none-any.whl (92 kB)
  Using cached yamlconf-0.2.4-py3-none-any.whl (9.1 kB)
  Using cached stopit-1.1.2.tar.gz (18 kB)
  Using cached revscoring-2.6.9-py2.py3-none-any.whl (391 kB)
  Using cached mwapi-0.5.1-py2.py3-none-any.whl (10 kB)
  Using cached celery-4.1.1-py2.py3-none-any.whl (394 kB)
  Using cached sseclient-0.0.27.tar.gz (7.5 kB)
  Using cached billiard-3.5.0.5.tar.gz (150 kB)
  Using cached kombu-4.6.11-py2.py3-none-any.whl (184 kB)
  Using cached amqp-2.6.1-py2.py3-none-any.whl (48 kB)
  Using cac

    NOT AVAILABLE
  
  atlas_blas_threads_info:
  Setting PTATLAS=ATLAS
    libraries ptf77blas,ptcblas,atlas not found in ['C:\\Users\\amsuc\\anaconda3\\lib', 'C:\\', 'C:\\Users\\amsuc\\anaconda3\\libs']
    NOT AVAILABLE
  
  atlas_blas_info:
    libraries f77blas,cblas,atlas not found in ['C:\\Users\\amsuc\\anaconda3\\lib', 'C:\\', 'C:\\Users\\amsuc\\anaconda3\\libs']
    NOT AVAILABLE
  
  C:\Users\amsuc\anaconda3\lib\site-packages\numpy\distutils\system_info.py:1989: UserWarning:
      Optimized (vendor) Blas libraries are not found.
      Falls back to netlib Blas library which has worse performance.
      A better performance should be easily gained by switching
      Blas library.
    if self._calc_info(blas):
  blas_info:
    libraries blas not found in ['C:\\Users\\amsuc\\anaconda3\\lib', 'C:\\', 'C:\\Users\\amsuc\\anaconda3\\libs']
    NOT AVAILABLE
  
  C:\Users\amsuc\anaconda3\lib\site-packages\numpy\distutils\system_info.py:1989: UserWarning:
      Blas (http://www.netlib

  copying sklearn\model_selection\tests\test_search.py -> build\lib.win-amd64-3.8\sklearn\model_selection/tests
  copying sklearn\model_selection\tests\test_split.py -> build\lib.win-amd64-3.8\sklearn\model_selection/tests
  copying sklearn\model_selection\tests\test_validation.py -> build\lib.win-amd64-3.8\sklearn\model_selection/tests
  copying sklearn\model_selection\tests\__init__.py -> build\lib.win-amd64-3.8\sklearn\model_selection/tests
  creating build\lib.win-amd64-3.8\sklearn\neural_network
  copying sklearn\neural_network\multilayer_perceptron.py -> build\lib.win-amd64-3.8\sklearn\neural_network
  copying sklearn\neural_network\rbm.py -> build\lib.win-amd64-3.8\sklearn\neural_network
  copying sklearn\neural_network\_base.py -> build\lib.win-amd64-3.8\sklearn\neural_network
  copying sklearn\neural_network\_stochastic_optimizers.py -> build\lib.win-amd64-3.8\sklearn\neural_network
  copying sklearn\neural_network\__init__.py -> build\lib.win-amd64-3.8\sklearn\neural_network


In [ ]:
from ores import api

ModuleNotFoundError: No module named 'ores'